https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/

# Imports

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [3]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

In [4]:
PATH = 'data/nyc-taxi/'

The entire dataset has about 55,423,857 rows

# Load clean dataframe (from v04a earlier version)

In [5]:
%%time
train_df = pd.read_feather('tmp/taxi-train-chkpt3')

Wall time: 8.63 s


In [6]:
test_df = pd.read_feather('tmp/taxi-test-chkpt3')

# Split Training-Validation Data

In [7]:
n_valid = 10_000  # same as Kaggle's test set size. Should we stick to it?

In [8]:
%time X, y, nas = proc_df(train_df, 'fare_amount')

Wall time: 24.1 s


In [9]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [10]:
n_trn = len(X)-n_valid

X_trn, X_val = split_vals(X, n_trn)
y_trn, y_val = split_vals(y, n_trn)

In [11]:
X_trn.shape, X_val.shape, y_trn.shape, y_val.shape

((54057064, 22), (10000, 22), (54057064,), (10000,))

# Train

## Helper function to calculate evaluation metric

In [12]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):  
    
    # in scikit-learn, random forest regressor,score is the r-squared value
    trn_scr = m.score(X_trn, y_trn)
    val_scr = m.score(X_val, y_val)
    
    # RMSE is what we are more interested in
    trn_rmse = rmse(m.predict(X_trn), y_trn)
    val_rmse = rmse(m.predict(X_val), y_val)
    if hasattr(m, 'oob_score_'): oob_scr = m.oob_score_
        
    res = f"Training-Score:   {trn_scr:.5f}\tTraining-rmse:   {trn_rmse:.5f}\n"
    res += f"Validation-score: {val_scr:.5f}\tValidation-rmse: {val_rmse:.5f}"   
    if hasattr(m, 'oob_score_'): res += f"\nOOB-score:\t  {oob_scr:.5f}"
        
    print(res)

## Use a subset of sample

In [13]:
subsample = 10_000_000

X_sub, y_sub, nas = proc_df(train_df, 'fare_amount', subset=subsample, na_dict=nas)
X_trn, _ = split_vals(X_sub, subsample-n_valid)
y_trn, _ = split_vals(y_sub, subsample-n_valid)

In [14]:
X_trn.shape, X_val.shape, y_trn.shape, y_val.shape 

((9990000, 22), (10000, 22), (9990000,), (10000,))

In [ ]:
m = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
%time m.fit(X_trn, y_trn)

In [ ]:
%time print_score(m)

# Predict on test data

In [ ]:
# We need 'key' for the test submission file
test_df_raw = pd.read_csv(f'{PATH}test.csv', usecols=['key'])

In [ ]:
test_y_predictions = m.predict(test_df)

# Submit

In [ ]:
from datetime import datetime as dt

In [ ]:
outdir = 'submissions'
os.makedirs(outdir, exist_ok=True)
    
cur_dt = dt.now().strftime("%Y%m%d_%H%M%S")
desc = 'v05_RF_' # description of submission for reference
subm_fn =  desc + cur_dt + '.csv'
subm_path = f'{outdir}/{subm_fn}'

# Write the predictions to a CSV file which we can submit to the competition.
submission = pd.DataFrame(
    {'key': test_df_raw.key, 'fare_amount': test_y_predictions},
    columns = ['key', 'fare_amount'])
submission.to_csv(subm_path, index = False)

subm_path

Kaggle Score: 3.07713,3.07349

# Save model

In [ ]:
model_fn_save = 'models/<>'

In [ ]:
%time pickle.dump(m, open(model_fn_save, 'wb'))

# Load Model

In [ ]:
model_fn_open = 'models/<>'

In [ ]:
%time m = pickle.load(open(model_fn_open, 'rb'))